<a href="https://colab.research.google.com/github/FREYDER18/PARCIAL-2025-1/blob/main/PROYECTO_SyS_WIFI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------------------------------------------
# 📦 Instalación de librerías necesarias
# ---------------------------------------------
!pip install streamlit
!pip install streamlit-option-menu
!pip install pyngrok  # No lo usaremos, pero útil si decides cambiar en el futuro

# Descargar Cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


In [ ]:
%%writefile app.py
import streamlit as st
from streamlit_option_menu import option_menu

# -------------------------------------------
# Configuración general del dashboard
# -------------------------------------------
st.set_page_config(
    page_title="Proyecto Final - Sistemas de Comunicaciones Digitales",
    layout="wide"
)

# -------------------------------------------
# Menú lateral de navegación entre fases
# -------------------------------------------
with st.sidebar:
    selected = option_menu(
        menu_title="📚 Navegación",
        options=[
            "🏠 Home",
            "🔄 Fase 1: Señal compuesta y análisis espectral",
            "🔄 Fase 2: Señales I/Q",
            "🔄 Fase 3: Modulación QAM-16",
            "🔄 Fase 4: OFDM + Ruido AWGN",
            "✅ Resultado Final"
        ],
        icons=[
            "house", "bar-chart", "activity",
            "diagram-3", "soundwave", "check2-circle"
        ],
        default_index=0
    )

# -------------------------------------------
# Pantalla de inicio del dashboard
# -------------------------------------------
if selected == "🏠 Home":
    st.title("📡 Proyecto Final - Sistemas de Comunicaciones Digitales")

    st.markdown(
        """
        Este dashboard simula un sistema completo de comunicaciones digitales, desde la generación de señales, modulación QAM, transmisión mediante OFDM, hasta la visualización de constelaciones y el efecto del ruido blanco (AWGN).

        Cada fase del sistema se puede visualizar de forma interactiva, permitiendo al usuario entender el proceso completo de una transmisión digital moderna.

        ### 👨‍💻 Integrantes del grupo:
        - **Juan Pablo Vargas Córdoba** - 1076904755
        - **Santiago Burgos Salazar** - 1121821274
        - **Luis Mateo Morales Rosero** - 1004620767

        ---
        **💡 Consejo:** Usa el menú de la izquierda para navegar por cada fase del sistema.
        """
    )


In [ ]:
%%writefile -a app.py
# -----------------------------------------
# 📡 Fase 1: Señal compuesta y análisis espectral
# -----------------------------------------
if selected == "🔄 Fase 1: Señal compuesta y análisis espectral":
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.signal import butter, lfilter

    st.header("📡 Fase 1: Dominio de la Frecuencia (Simulación Básica)")

    st.markdown("### 🔊 Generación de Señal Sintética y Filtro Paso Bajo")

    # Parámetros ajustables
    f1 = st.slider("Frecuencia 1 (Hz)", 10, 100, 50)
    f2 = st.slider("Frecuencia 2 (Hz)", 100, 300, 120)
    f3 = st.slider("Frecuencia 3 (Hz)", 200, 500, 300)
    fcorte = st.slider("Frecuencia de Corte del Filtro (Hz)", 50, 400, 150)
    fs = 1000
    t = np.linspace(0, 1, fs, endpoint=False)

    # Señal compuesta
    x = (np.sin(2*np.pi*f1*t) +
         0.5*np.sin(2*np.pi*f2*t) +
         0.2*np.sin(2*np.pi*f3*t))

    # Filtro pasa-bajo Butterworth
    order = 4
    b, a = butter(order, fcorte / (fs/2), btype='low')
    x_filtrada = lfilter(b, a, x)

    # FFT
    X = np.fft.rfft(x)
    X_mag = 2/len(x) * np.abs(X)
    Xf = np.fft.rfft(x_filtrada)
    Xf_mag = 2/len(x) * np.abs(Xf)
    f = np.fft.rfftfreq(len(x), 1/fs)

    # Señal en el tiempo
    st.subheader("📈 Señales en el Dominio del Tiempo")
    fig1, ax1 = plt.subplots()
    ax1.plot(t, x, label='Original')
    ax1.plot(t, x_filtrada, label='Filtrada', color='red')
    ax1.set_title("Señal Original vs Filtrada")
    ax1.set_xlabel("Tiempo [s]")
    ax1.set_ylabel("Amplitud")
    ax1.grid(True)
    ax1.legend()
    st.pyplot(fig1)

    # Espectros
    st.subheader("📊 Espectros de Frecuencia")
    fig2, ax2 = plt.subplots()
    ax2.plot(f, X_mag, label='Original')
    ax2.plot(f, Xf_mag, label='Filtrada', color='red')
    ax2.set_title("Espectro de la Señal (FFT)")
    ax2.set_xlabel("Frecuencia [Hz]")
    ax2.set_ylabel("Amplitud")
    ax2.grid(True)
    ax2.legend()
    st.pyplot(fig2)


In [ ]:
%%writefile -a app.py
# -----------------------------------------------------------
# 🧠 FASE 2: Construcción de Señales I/Q con Transformada de Hilbert
# -----------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert

# Página de Fase 2
if selected == "🔄 Fase 2: Señales I/Q":
    st.header("🔄 Fase 2: Construcción de Señales I/Q")

    # Parámetros ajustables
    f0 = st.slider("Frecuencia de la señal I(t) [Hz]", 1, 50, 10)
    fs = st.slider("Frecuencia de muestreo [Hz]", 200, 2000, 1000)
    duracion = st.slider("Duración de la señal [s]", 1, 5, 1)

    # Tiempo
    t = np.linspace(0, duracion, int(fs*duracion), endpoint=False)

    # Señal I(t)
    I_t = np.sin(2 * np.pi * f0 * t)

    # Transformada de Hilbert para obtener Q(t)
    x_a = hilbert(I_t)               # Señal analítica: I(t) + jQ(t)
    Q_t = np.imag(x_a)               # Parte imaginaria = señal en cuadratura
    modulo = np.abs(x_a)             # Módulo de la señal compleja

    # 🔹 Mostrar señal I/Q y módulo
    st.subheader("📈 Señales I(t), Q(t) y módulo")
    fig1, ax1 = plt.subplots(figsize=(10, 4))
    ax1.plot(t, I_t, label="I(t) - Parte real", color='blue')
    ax1.plot(t, Q_t, label="Q(t) - Parte imaginaria", color='orange')
    ax1.plot(t, modulo, '--', label="Módulo", color='green')
    ax1.set_xlabel("Tiempo [s]")
    ax1.set_ylabel("Amplitud")
    ax1.set_title("Señales I(t), Q(t) y Módulo")
    ax1.legend()
    ax1.grid(True)
    st.pyplot(fig1)

    # 🔹 Análisis espectral con FFT
    N = len(t)
    freqs = np.fft.fftfreq(N, 1/fs)
    freqs_shifted = np.fft.fftshift(freqs)

    X_I = np.fft.fft(I_t) / N
    X_Q = np.fft.fft(Q_t) / N
    X_mod = np.fft.fft(modulo) / N

    X_I_mag = 2 * np.abs(np.fft.fftshift(X_I))
    X_Q_mag = 2 * np.abs(np.fft.fftshift(X_Q))
    X_mod_mag = 2 * np.abs(np.fft.fftshift(X_mod))

    # 🔹 Mostrar análisis espectral
    st.subheader("🔍 Análisis espectral (FFT)")
    fig2, ax2 = plt.subplots(figsize=(10, 4))
    ax2.plot(freqs_shifted, X_I_mag, label='FFT I(t)', color='blue')
    ax2.plot(freqs_shifted, X_Q_mag, label='FFT Q(t)', color='orange')
    ax2.plot(freqs_shifted, X_mod_mag, label='FFT módulo', color='green')
    ax2.set_xlim(-100, 300)
    ax2.set_xlabel("Frecuencia [Hz]")
    ax2.set_ylabel("Amplitud")
    ax2.set_title("Espectro de las señales I/Q y módulo")
    ax2.legend()
    ax2.grid(True)
    st.pyplot(fig2)


In [ ]:
%%writefile app.py -a
# ---------------------------------------------
# 🧩 FASE 3 – MODULACIÓN QAM-16 Y BANDA PASANTE
# ---------------------------------------------

elif selected == "🔄 Fase 3: Modulación QAM-16":

    with st.container():
        st.header("📡 Fase 3: Modulación QAM-16 y Banda Pasante")

        # Parámetros ajustables
        num_symbols = st.slider("Cantidad de símbolos a transmitir", 100, 1000, 400, step=100)
        fc = st.slider("Frecuencia de portadora [Hz]", 500, 5000, 1000, step=500)
        fs = st.slider("Frecuencia de muestreo [Hz]", 5000, 50000, 10000, step=5000)

        # Bits por símbolo (QAM-16 -> 4 bits)
        M = 16
        k = int(np.log2(M))

        # Generar datos aleatorios
        bits = np.random.randint(0, 2, num_symbols * k)
        symbols = bits.reshape((-1, k))

        # Diccionario de mapeo I/Q para 16-QAM
        mapping = {
            (0,0,0,0): (-3,-3), (0,0,0,1): (-3,-1), (0,0,1,1): (-3, 1), (0,0,1,0): (-3, 3),
            (0,1,0,0): (-1,-3), (0,1,0,1): (-1,-1), (0,1,1,1): (-1, 1), (0,1,1,0): (-1, 3),
            (1,1,0,0): ( 1,-3), (1,1,0,1): ( 1,-1), (1,1,1,1): ( 1, 1), (1,1,1,0): ( 1, 3),
            (1,0,0,0): ( 3,-3), (1,0,0,1): ( 3,-1), (1,0,1,1): ( 3, 1), (1,0,1,0): ( 3, 3),
        }

        # Asignar coordenadas I/Q
        I = []
        Q = []
        for b in symbols:
            i, q = mapping[tuple(b)]
            I.append(i)
            Q.append(q)

        # Normalización para que la potencia promedio sea 1
        I = np.array(I) / np.sqrt(10)
        Q = np.array(Q) / np.sqrt(10)

        # Mostrar diagrama de constelación
        fig1, ax1 = plt.subplots()
        ax1.scatter(I, Q, color='purple', alpha=0.6)
        ax1.axhline(0, color='gray', linewidth=0.5)
        ax1.axvline(0, color='gray', linewidth=0.5)
        ax1.set_title("📍 Diagrama de Constelación 16-QAM")
        ax1.set_xlabel("I (In-Phase)")
        ax1.set_ylabel("Q (Quadrature)")
        ax1.grid(True)
        ax1.set_aspect('equal')
        st.pyplot(fig1)

        # 🛰️ Construcción señal modulada
        samples_per_symbol = int(fs / fc)
        t_symbol = np.linspace(0, 1/fs * samples_per_symbol, samples_per_symbol, endpoint=False)
        s = []

        for i_sym, q_sym in zip(I, Q):
            st_sample = i_sym * np.cos(2*np.pi*fc*t_symbol) - q_sym * np.sin(2*np.pi*fc*t_symbol)
            s.extend(st_sample)

        s = np.array(s)

        # 🔍 Visualizar señal modulada
        st.subheader("📶 Señal QAM-16 en Banda Pasante")
        duration_to_plot = st.slider("Duración a visualizar [ms]", 1, 10, 5)
        N = int(fs * (duration_to_plot / 1000))
        t_total = np.linspace(0, N/fs, N)

        fig2, ax2 = plt.subplots()
        ax2.plot(t_total, s[:N])
        ax2.set_title(f"🧪 Fragmento de señal modulada (primeros {duration_to_plot} ms)")
        ax2.set_xlabel("Tiempo [s]")
        ax2.set_ylabel("Amplitud")
        ax2.grid(True)
        st.pyplot(fig2)


In [ ]:
%%writefile -a app.py

# ---------------------------------------------------------------
# 🧩 FASE 4 - Canal con ruido, demodulación y constelación recibida
# ---------------------------------------------------------------

elif selected == "🔄 Fase 4: OFDM + Ruido AWGN":

    st.header("🌀 Fase 4: Canal con Ruido, Demodulación y Constelación Recibida")

    # 🎛️ Parámetros interactivos: valores de SNR en dB
    snr_db = st.slider("📶 Selecciona el SNR [dB]", min_value=0, max_value=40, value=20, step=5)

    # 🔧 Parámetros de modulación
    M = 16                                  # QAM-16
    k = int(np.log2(M))                     # 4 bits por símbolo
    symbols = np.random.randint(0, 2, 1000 * k).reshape(-1, k)

    # 🔁 Mapeo QAM 16 (no Gray, potencia normalizada)
    mapping = {
        (0,0,0,0): (-3,-3), (0,0,0,1): (-3,-1), (0,0,1,1): (-3, 1), (0,0,1,0): (-3, 3),
        (0,1,0,0): (-1,-3), (0,1,0,1): (-1,-1), (0,1,1,1): (-1, 1), (0,1,1,0): (-1, 3),
        (1,1,0,0): ( 1,-3), (1,1,0,1): ( 1,-1), (1,1,1,1): ( 1, 1), (1,1,1,0): ( 1, 3),
        (1,0,0,0): ( 3,-3), (1,0,0,1): ( 3,-1), (1,0,1,1): ( 3, 1), (1,0,1,0): ( 3, 3),
    }

    # 📡 Mapeamos los símbolos a coordenadas I/Q
    I = []
    Q = []
    for b in symbols:
        i, q = mapping[tuple(b)]
        I.append(i)
        Q.append(q)

    # ⚖️ Normalización para potencia unitaria
    I = np.array(I) / np.sqrt(10)
    Q = np.array(Q) / np.sqrt(10)
    tx_symbols = I + 1j*Q   # Señal transmitida como números complejos

    # 🌪️ Función para agregar ruido blanco AWGN
    def add_awgn(signal, snr_db):
        snr_linear = 10**(snr_db / 10)
        signal_power = np.mean(np.abs(signal)**2)
        noise_power = signal_power / snr_linear
        noise = np.sqrt(noise_power/2) * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
        return signal + noise

    # 🎯 Canal con ruido
    rx_symbols = add_awgn(tx_symbols, snr_db)

    # 🎨 Visualización de la constelación recibida
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(np.real(rx_symbols), np.imag(rx_symbols), color='crimson', alpha=0.4, label="Rx con ruido")
    ax.axhline(0, color='gray', linewidth=0.5)
    ax.axvline(0, color='gray', linewidth=0.5)
    ax.set_title(f"📡 Constelación recibida con SNR = {snr_db} dB")
    ax.set_xlabel("I (In-phase)")
    ax.set_ylabel("Q (Quadrature)")
    ax.set_aspect('equal')
    ax.grid(True)
    st.pyplot(fig)



In [ ]:
%%writefile -a app.py

# ------------------------------------------
# ✅ FASE FINAL - Resultado del sistema completo
# ------------------------------------------
elif selected == "✅ Resultado Final":
    st.header("📡 Fase Final - Resultado del Sistema Completo")

    st.markdown("""
    En esta sección visualizamos el resultado de la **transmisión digital completa** usando modulación QAM-16.
    Se incluye ruido en el canal (AWGN) y se representa la constelación que vería el receptor.
    Esto simula cómo funciona la transmisión de datos en **WiFi y 5G**.

    **Puedes ajustar el nivel de ruido (SNR)** para ver cómo se afecta la calidad de la transmisión.
    """)

    # Slider SNR
    snr_db_final = st.slider("🛠️ Nivel de ruido SNR [dB]", min_value=5, max_value=40, value=20, step=1)

    # Generar símbolos
    M = 16
    k = int(np.log2(M))
    bits = np.random.randint(0, 2, 1000 * k)
    symbols = bits.reshape((-1, k))

    mapping = {
        (0,0,0,0): (-3,-3), (0,0,0,1): (-3,-1), (0,0,1,1): (-3, 1), (0,0,1,0): (-3, 3),
        (0,1,0,0): (-1,-3), (0,1,0,1): (-1,-1), (0,1,1,1): (-1, 1), (0,1,1,0): (-1, 3),
        (1,1,0,0): ( 1,-3), (1,1,0,1): ( 1,-1), (1,1,1,1): ( 1, 1), (1,1,1,0): ( 1, 3),
        (1,0,0,0): ( 3,-3), (1,0,0,1): ( 3,-1), (1,0,1,1): ( 3, 1), (1,0,1,0): ( 3, 3),
    }

    I = []
    Q = []
    for b in symbols:
        i, q = mapping[tuple(b)]
        I.append(i)
        Q.append(q)

    I = np.array(I) / np.sqrt(10)
    Q = np.array(Q) / np.sqrt(10)
    symbols_clean = I + 1j * Q

    # Función ruido
    def awgn(signal, snr_db):
        snr_linear = 10**(snr_db / 10)
        signal_power = np.mean(np.abs(signal)**2)
        noise_power = signal_power / snr_linear
        noise = np.sqrt(noise_power/2) * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
        return signal + noise

    symbols_noisy_final = awgn(symbols_clean, snr_db_final)

    # Plot constelación
    fig_final, ax_final = plt.subplots(figsize=(6, 6))
    ax_final.scatter(np.real(symbols_noisy_final), np.imag(symbols_noisy_final), alpha=0.6, color='purple')
    ax_final.axhline(0, color='gray', linewidth=0.5)
    ax_final.axvline(0, color='gray', linewidth=0.5)
    ax_final.set_title(f'Constelación QAM-16 con ruido (SNR = {snr_db_final} dB)')
    ax_final.set_xlabel('I (En fase)')
    ax_final.set_ylabel('Q (En cuadratura)')
    ax_final.grid(True)
    ax_final.set_aspect('equal')
    st.pyplot(fig_final)

    st.markdown("""
    ---
    ### 🎯 Finalidad del Proyecto
    Esta simulación muestra cómo se codifican, modulan y transmiten datos digitales en sistemas de comunicación modernos.

    - **QAM-16** permite enviar 4 bits por símbolo (alta eficiencia espectral).
    - **OFDM** mejora la robustez en canales multipath (como en WiFi y 5G).
    - **Ruido AWGN** simula un canal real y permite estudiar la calidad de recepción.

    ✅ Este tipo de procesamiento es el **núcleo de tecnologías inalámbricas modernas** como:
    - **WiFi 802.11ac/ax**
    - **5G NR (New Radio)**
    - **LTE (4G)**
    """)


In [ ]:
import os
import threading
import time

# ✅ Iniciar Streamlit en segundo plano
def run_streamlit():
    os.system("streamlit run app.py &")

# 🧵 Lanzar Streamlit en hilo separado
threading.Thread(target=run_streamlit).start()

# ⏱️ Esperar que arranque
time.sleep(5)

# 🌐 Ejecutar túnel Cloudflared
print("🌐 Iniciando Cloudflared...")
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate
